In [1]:
# === U3 eigen-analysis with aligned sliders + degree boxes (no QuTiP) ===
# Responsive layout: single-row angle controls, compact widths, capped app width.

from datetime import date
import numpy as np
from numpy.linalg import eig
import matplotlib.pyplot as plt
from ipywidgets import (
    FloatSlider, FloatText, IntSlider, Dropdown, Button,
    HBox, VBox, Output, Layout, HTMLMath
)
from IPython.display import display, clear_output, Markdown

# ------------------ Display / figure tuning ------------------
plt.rcParams['figure.dpi'] = 110
plt.rcParams['figure.figsize'] = (6.6, 4.0)
plt.rcParams['figure.autolayout'] = True

VERSION = "v1.1"
DATE = (date.today().isoformat())

# ------------------ Math core ------------------
def u3(theta, phi, lam):
    c = np.cos(theta/2.0); s = np.sin(theta/2.0)
    return np.array([[c, -np.exp(1j*lam)*s],
                     [np.exp(1j*phi)*s, np.exp(1j*(phi+lam))*c]], dtype=complex)

def eigpairs_unitary(U, sort_by_phase=True, rtol=1e-12):
    evals, evecs = eig(U)
    # normalize eigenvectors (columns)
    for k in range(2):
        n = np.linalg.norm(evecs[:, k])
        if n: evecs[:, k] /= n
    # handle (near) degeneracy robustly
    if np.allclose(evals[0], evals[1], rtol=rtol, atol=rtol):
        ev = np.mean(evals)
        evals = np.array([ev, ev], dtype=complex)
        evecs = np.eye(2, dtype=complex)
    # sort by phase angle for stable display
    if sort_by_phase:
        order = np.argsort(np.angle(evals))
        evals = evals[order]; evecs = evecs[:, order]
    return evals, evecs

def probs_from_plus_z(evecs):
    # In {|+z>, |-z>} basis, amplitude on |+z> is row 0
    return np.abs(evecs[0, :])**2

def cfmt(z, sig=2):
    a, b = np.real(z), np.imag(z)
    return f"{a:.{sig}g}{'+' if b>=0 else '-'}{abs(b):.{sig}g}i"

# ------------------ Responsive widths ------------------
APP_MAX_W = '950px'  # cap total width so it doesn't sprawl on Linux
LEFT_W    = '300px'  # fixed width for left control column
COL_W     = '240px'  # each angle column (slider + degree box)
SLIM_W    = '200px'  # slider width
DEG_W     = '120px'  # degree FloatText width
LBL_W     = '28px'   # label width for descriptions

# ------------------ Widgets ------------------
preset = Dropdown(
    description='Preset',
    options=[('Custom','custom'),('Identity (I)','I'),('σ_x','X'),('σ_y','Y'),('σ_z','Z')],
    value='custom',
    layout=Layout(width='100%')
)
preset.style={'description_width':'70px'}

apply_btn = Button(description='Apply', layout=Layout(width='96px'))

nruns = IntSlider(description='Ntrial', min=1, max=200000, step=1, value=1000,
                  continuous_update=False, layout=Layout(width='100%'))
nruns.style={'description_width':'70px'}

seed  = IntSlider(description='Seed',  min=0, max=2**31-1, step=1, value=1234,
                  continuous_update=False, layout=Layout(width='100%'))
seed.style={'description_width':'70px'}

# Sliders (radians) -- compact
theta = FloatSlider(description='θ', min=0.0, max=np.pi, step=0.001, value=0.0,
                    readout_format='.4f', continuous_update=False,
                    layout=Layout(width=SLIM_W))
theta.style={'description_width': LBL_W}

phi = FloatSlider(description='φ', min=0.0, max=2*np.pi, step=0.001, value=0.0,
                  readout_format='.4f', continuous_update=False,
                  layout=Layout(width=SLIM_W))
phi.style={'description_width': LBL_W}

lam = FloatSlider(description='λ', min=0.0, max=2*np.pi, step=0.001, value=0.0,
                  readout_format='.4f', continuous_update=False,
                  layout=Layout(width=SLIM_W))
lam.style={'description_width': LBL_W}

# Degree boxes (synced) -- compact
theta_deg = FloatText(description='θ°', value=0.0, step=0.1, layout=Layout(width=DEG_W))
phi_deg   = FloatText(description='φ°', value=0.0, step=0.1, layout=Layout(width=DEG_W))
lam_deg   = FloatText(description='λ°', value=0.0, step=0.1, layout=Layout(width=DEG_W))
for box in (theta_deg, phi_deg, lam_deg):
    box.style={'description_width': LBL_W}

# Outputs: flex 1:1 so they share row nicely
out_text = Output(layout=Layout(
    border='1px solid #ddd', padding='8px',
    width='auto', height='380px', overflow_y='auto',
    flex='1 1 0', min_width='0'
))
out_plot = Output(layout=Layout(
    border='1px solid #ddd', padding='8px',
    width='auto', height='380px',
    flex='1 1 0', min_width='0'
))

# ------------------ Wiring / behavior ------------------
def set_preset(tag):
    if tag=='I':
        theta.value=0.0; phi.value=0.0; lam.value=0.0
    elif tag=='Z':
        theta.value=0.0; phi.value=0.0; lam.value=np.pi
    elif tag=='X':
        theta.value=np.pi; phi.value=0.0; lam.value=np.pi
    elif tag=='Y':
        theta.value=np.pi; phi.value=np.pi/2; lam.value=np.pi/2

apply_btn.on_click(lambda _:
    set_preset(preset.value) if preset.value!='custom' else None
)

def _sync_deg_from_sliders(_=None):
    theta_deg.value = float(f"{np.degrees(theta.value):.4f}")
    phi_deg.value   = float(f"{np.degrees(phi.value):.4f}")
    lam_deg.value   = float(f"{np.degrees(lam.value):.4f}")

def _coerce_angle(x, lo, hi):
    if np.isnan(x) or np.isinf(x): return lo
    return float(min(max(x, lo), hi))

def _sync_sliders_from_deg(ch):
    src = ch['owner']
    if src is theta_deg: theta.value = _coerce_angle(np.radians(theta_deg.value), 0, np.pi)
    elif src is phi_deg: phi.value   = _coerce_angle(np.radians(phi_deg.value),   0, 2*np.pi)
    elif src is lam_deg: lam.value   = _coerce_angle(np.radians(lam_deg.value),   0, 2*np.pi)

def render(*_):
    th, ph, la = theta.value, phi.value, lam.value
    U = u3(th, ph, la)
    evals, evecs = eigpairs_unitary(U)
    probs = probs_from_plus_z(evecs); probs /= probs.sum()

    rng = np.random.default_rng(seed.value)
    counts = rng.multinomial(nruns.value, probs)
    freqs = counts / counts.sum()

    with out_text:
        clear_output(wait=True)
        print(f"Version: {VERSION}  |  Date: {DATE}")
        display(Markdown(r"**Unitary** $U(\theta,\phi,\lambda)$ with angles (rad):"))
        display(Markdown(fr"$\theta$ = {th:.4f},  $\phi$ = {ph:.4f},  $\lambda$ = {la:.4f}"))
        print("U =")
        for r in range(2):
            print("  [ " + " , ".join(cfmt(U[r,c], sig=4) for c in range(2)) + " ]")
        print("\nEigenvalue : eigenvector (rows):")
        for k in range(2):
            vec = "[" + " , ".join(cfmt(evecs[r,k], sig=4) for r in range(2)) + "]"
            print(f"{k}:    {cfmt(evals[k], sig=4)} : {vec}")
        display(Markdown(r"Probabilities $P(0)=|\langle +z|0\rangle|^2$ and $P(1)=|\langle -z|0\rangle|^2$:"))
        display(Markdown(fr"&nbsp;&nbsp; $P_{{\mathrm{{theo}}}}(0)$ = {probs[0]:.4f}; "
                         fr"$P_{{\mathrm{{theo}}}}(1)$ = {probs[1]:.4f}; "
                         fr"$\mathrm{{Sum}}$ = {(probs[0]+probs[1]):.2f}"))
        display(Markdown(fr"Sampling with Ntrial = {nruns.value}; Seed = {seed.value}:"))
        display(Markdown(fr"&nbsp;&nbsp; counts(0) = {int(counts[0])},  counts(1) = {int(counts[1])}"))
        display(Markdown(fr"&nbsp;&nbsp; $P_{{\mathrm{{samp}}}}(0)$ = {freqs[0]:.4g}, "
                         fr"$P_{{\mathrm{{samp}}}}(1)$ = {freqs[1]:.4g}"))

    with out_plot:
        clear_output(wait=True)
        labels = ['P(+z)','P(-z)','P(0)','P(1)']
        heights = [1.0, 0.0, freqs[0], freqs[1]]
        x = np.arange(len(labels))
        fig, ax = plt.subplots()
        ax.bar(x, heights)
        ax.set_xticks(x, labels)
        ax.set_ylim(0, 1.2)  # headroom
        ax.set_ylabel("Probability")
        ax.set_title("Histogram: Input + Outcome Probabilities", y=1.02, pad=10)
        # text overlays
        for i, lab in enumerate(labels):
            if lab in ('P(0)','P(1)'):
                idx = 0 if lab=='|0>' else 1
                y0 = heights[i]
                ax.text(i, y0 + 0.12, rf"$P_{{\mathrm{{theo}}}}={probs[idx]:.4g}$", ha='center', va='bottom')
                ax.text(i, y0 + 0.07, rf"$P_{{\mathrm{{samp}}}}={freqs[idx]:.4g}$", ha='center', va='bottom')
                ax.text(i, y0 + 0.02, rf"$N={counts[idx]}$",               ha='center', va='bottom')
        ax.grid(True, axis='y', alpha=0.3)

        fig.text(0.32, 0.01, "Input", ha='center', va='center', fontsize=12, fontweight='bold')
        fig.text(0.75, 0.01, "Output", ha='center', va='center', fontsize=12, fontweight='bold')
        
        plt.show()
        print(f"Ntrial: {nruns.value}")

# ------------------ Observers ------------------
for w in (theta, phi, lam, preset, nruns, seed):
    w.observe(render, names='value')

for s in (theta, phi, lam):
    s.observe(_sync_deg_from_sliders, names='value')
_sync_deg_from_sliders()

for b in (theta_deg, phi_deg, lam_deg):
    b.observe(_sync_sliders_from_deg, names='value')

# ------------------ Layout ------------------
# Left column (fixed width)
left_controls = VBox(
    [HBox([preset, apply_btn], layout=Layout(gap='8px')),
     nruns, seed],
    layout=Layout(width=LEFT_W)
)

# Three compact angle columns (ONE ROW)
theta_col = VBox([theta, theta_deg], layout=Layout(width=COL_W))
phi_col   = VBox([phi,   phi_deg],   layout=Layout(width=COL_W))
lam_col   = VBox([lam,   lam_deg],   layout=Layout(width=COL_W))

angles_row = HBox([theta_col, phi_col, lam_col],
                  layout=Layout(gap='12px', width='auto'))

# Top row: left controls + one-row angles
top_row = HBox(
    [left_controls, angles_row],
    layout=Layout(justify_content='flex-start',
                  align_items='flex-start', gap='16px', width='100%')
)

# Bottom: outputs share width equally
bottom_outputs = HBox(
    [out_text, out_plot],
    layout=Layout(gap='12px', align_items='stretch', width='100%')
)

# App root with a max-width cap (prevents over-wide UI on Linux)
app_root = VBox([top_row, bottom_outputs],
                layout=Layout(width='100%', max_width=APP_MAX_W))

# ------------------ Go ------------------
render()
display(app_root)
